In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [ ]:
# This dataset was found at https://osu.app.box.com/v/us-accidents-dec19 and unzipped with 7-Zip

In [2]:
df = pd.read_csv('C:/Users/ggibs/data/US_Accidents_Dec19.csv', parse_dates = ['Start_Time', 'End_Time'])  

In [3]:
df['Month'] = df['Start_Time'].dt.month

In [4]:
# Zipcode is null in only 0.03% of records, remove null zipcode rows
df = df[df['Zipcode'].notna()]

In [5]:
# Shorten zipcode for larger areas
df['Zip2'] = df['Zipcode'].str[:2]

In [6]:
# Group data by Zipcode2 and Month for regional weather similarities
df_group = df.groupby(['Zip2', 'Month'], as_index = False)

In [7]:
# Calculate mean and std deviation of weather measurements by zip-month grouping
monthly_precip = df_group['Precipitation(in)'].agg(['mean', 'std']).reset_index()
monthly_chill = df_group['Wind_Chill(F)'].agg(['mean', 'std']).reset_index()
monthly_speed = df_group['Wind_Speed(mph)'].agg(['mean', 'std']).reset_index()

In [8]:
# Create an index by zip-month combo
monthly_precip['Zip2Month'] = monthly_precip['Zip2'] + monthly_precip['Month'].astype(str)
mp = monthly_precip[['Zip2Month', 'mean', 'std']].set_index('Zip2Month')

monthly_chill['Zip2Month'] = monthly_chill['Zip2'] + monthly_chill['Month'].astype(str)
mc = monthly_chill[['Zip2Month', 'mean', 'std']].set_index('Zip2Month')

monthly_speed['Zip2Month'] = monthly_speed['Zip2'] + monthly_speed['Month'].astype(str)
ms = monthly_speed[['Zip2Month', 'mean', 'std']].set_index('Zip2Month')

In [12]:
# Fill weather NaNs with above means and standard deviations
#for i in df.index:
for i in range(1000):
    row = df.loc[i,'Zip2'] + df.loc[i,'Month'].astype(str)
    try:
        if pd.isnull(df.loc[i, 'Precipitation(in)']):
            df.loc[i, 'Precipitation(in)'] = np.random.normal(mp.loc[row, 'mean'], mp.loc[row, 'std'], 1)
    except:
        continue
    try:
        if pd.isnull(df.loc[i, 'Wind_Chill(F)']):
            df.loc[i, 'Wind_Chill(F)'] = np.random.normal(mc.loc[row, 'mean'], mc.loc[row, 'std'], 1)
    except:
        continue
    try:
        if pd.isnull(df.loc[i, 'Wind_Speed(mph)']):
            df.loc[i, 'Wind_Speed(mph)'] = np.random.normal(ms.loc[row, 'mean'], ms.loc[row, 'std'], 1)
    except:
        continue
    else:
        continue